# Using Pycaret Tabular Playground Competition! #



In [ ]:
import pandas as pd
from pathlib import Path

data_dir = Path('../input/tabular-playground-series-sep-2021/')

df_train = pd.read_csv(
    data_dir / "train.csv",
    index_col='id',
    nrows=25000,  # comment this row to use the full dataset
)

FEATURES = df_train.columns[:-1]
TARGET = df_train.columns[-1]

df_train.head()

In [ ]:
!pip install pycaret[full]

**Import libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.classification import *

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

data = pd.concat([train, test])

In [ ]:
df_data = data.drop(columns = 'id')


In [ ]:
plt.figure(figsize = (12, 6))
missing_values = df_data.isnull().sum()[:-1]
sns.histplot(missing_values, color='violet');
plt.show()

In [ ]:
print('\n')
print('-------- Distribution of Missing values --------')
print('Min:', missing_values.min())
print('Max:', missing_values.max())
print('Mean:', missing_values.mean())
print('------------------------------------------------')

In [ ]:
df_data.describe

In [ ]:
train2 = df_data[:len(train)]
test2 = df_data[len(train):]
test2.drop(columns = 'claim', inplace = True)

In [ ]:
from pycaret.classification import *
clf = setup(data=train2, target = 'claim', session_id=42, log_experiment=True,numeric_imputation = 'mean',normalize = True,
        normalize_method = 'robust',fold = 3,
        silent= True,
        use_gpu=True,)

In [ ]:
cat = create_model(estimator = 'catboost', fold = 5)
    

In [ ]:
lgbm = create_model(estimator = 'lightgbm', fold = 5)


In [ ]:
xgb = create_model(estimator = 'xgboost', fold = 5)


In [ ]:
boosted_lgbm = ensemble_model(lgbm, method = 'Boosting')


In [ ]:
blender = blend_models(estimator_list = [cat,lgbm,xgb], fold=5)


In [ ]:
pred_holdout = predict_model(blender)

In [ ]:
final_model = finalize_model(blender)

In [ ]:
prep_pipe = get_config('prep_pipe')
prep_pipe.steps.append(['trained_model', final_model])
pred = prep_pipe.predict_proba(test2)

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
sub

In [ ]:
result=pred

In [ ]:
sub['claim'] = result[:, 1]
sub.to_csv('sub.csv', index = 0)